In [49]:
import numpy as np
import pandas as pd
import matplotlib as plt
import geopandas as gpd # conda install geopandas
from geopandas.tools import sjoin
import shapely
from shapely.geometry import Point, Polygon
import folium # conda install -c conda-forge folium
import json
import pickle
import geopy
from geopy.extra.rate_limiter import RateLimiter
import re
import requests

### Import data
We import the dataset containing the food inspection and drop all irrelevant and null columns.

In [129]:
data = pd.read_csv('data/food-inspections.csv').drop(['AKA Name', 'City', 'State', "Historical Wards 2003-2015", "Zip Codes", "Community Areas", "Census Tracts", "Wards", "Location"], axis=1)
data.head()

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
0,2320509,TAQUERIA EL DORADO,2694960.0,Restaurant,Risk 1 (High),2114 W LAWRENCE AVE,60625.0,2019-10-25T00:00:00.000,License Re-Inspection,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.968820,-87.682292
1,2320519,SALAM RESTAURANT,2002822.0,Restaurant,Risk 1 (High),4634-4636 N KEDZIE AVE,60625.0,2019-10-25T00:00:00.000,Complaint Re-Inspection,Pass,NaN,41.965719,-87.708538
2,2320421,THE NEW GRACE RESTAURANT,2698310.0,Restaurant,Risk 1 (High),4409 N BROADWAY,60640.0,2019-10-24T00:00:00.000,License,Not Ready,NaN,41.962104,-87.655204
3,2320368,GADS HILL CHILD CARE CENTER,2215799.0,Children's Services Facility,Risk 1 (High),2653 W OGDEN AVE,60608.0,2019-10-24T00:00:00.000,License,Pass,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.862273,-87.692703
4,2320389,"ALL THINGS ARE POSSIBLE FOR KIDS, INC.",2215965.0,Children's Services Facility,Risk 1 (High),4014 W CHICAGO AVE,60651.0,2019-10-24T00:00:00.000,License,Pass,54. GARBAGE & REFUSE PROPERLY DISPOSED; FACILI...,41.895468,-87.726665


# Complete the dataset

## Duplicates
Entries must be identified by their Inspection ID. Duplicate rows can simply be removed.

In [130]:
print("Number of duplicate inspection IDs: " + str(len(data[data.duplicated("Inspection ID")])))
data = data.drop_duplicates()
print("Number of duplicate inspection IDs after removal: " + str(len(data[data.duplicated("Inspection ID")])))

Number of duplicate inspection IDs: 196
Number of duplicate inspection IDs after removal: 0


## Deal with missing values

In [7]:
for col in data.columns:
    print("Number of entries without " + col + ":" + str(len(data[data[col].apply(pd.isnull)])))

Number of entries without Inspection ID:0
Number of entries without DBA Name:0
Number of entries without License #:17
Number of entries without Facility Type:4770
Number of entries without Risk:72
Number of entries without Address:0
Number of entries without Zip:50
Number of entries without Inspection Date:0
Number of entries without Inspection Type:1
Number of entries without Results:0
Number of entries without Violations:51636
Number of entries without Latitude:682
Number of entries without Longitude:682


### Address
Regardless of what it looks like in the missing values, there are some addresses that are missing in that they only contain a few spaces.

In [119]:
data[data['Address'].apply(str.isspace)]

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
69601,1763245,EAT N RUN CHICKEN AND FISH,2442868.0,Restaurant,Risk 1 (High),,NaN,2016-04-19T00:00:00.000,License,Not Ready,No data,NaN,NaN
189113,114452,Starfruit Cafe,2031651.0,NaN,Risk 2 (Medium),,60654.0,2010-05-05T00:00:00.000,License,Fail,No data,NaN,NaN
190589,60405,CASA CENTRAL LA POSADA,0.0,Restaurant,Risk 1 (High),,NaN,2010-04-05T00:00:00.000,Canvass,Pass,37. TOILET ROOM DOORS SELF CLOSING: DRESSING R...,NaN,NaN


### Latitude and longitude
There are entries that are missing coordinate data, which is needed to put them on the map.
We see that all entries have address data, so we try to fill in the null values for the coordinates by using Geopy that uses geocoders to get coordinates based on addresses. Through testing we found that using ArcGIS as a geocoder gives us a perfect result.

In [ ]:
locator = geopy.geocoders.ArcGIS(user_agent='myGeocoder')
data_wo_latlong = data[data['Latitude'].apply(np.isnan)].groupby("DBA Name").first()
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
lats = pd.Series(index=data_wo_latlong.index)
longs = pd.Series(index=data_wo_latlong.index)
for name, address in list(zip(data_wo_latlong.index, data_wo_latlong['Address'])):
    if address == "":
        print('Empty address for ' + name + 'ignored')
        continue
    location = geocode(address + ", Chicago, United States")
    if location is not None:
        lats[name] = location.latitude
        longs[name] = location.longitude
    else:
        print('Did not find location for address: ' + address)

for name, lat, long in zip(lats.index, lats, longs):
    data['Latitude'] = np.where(data['DBA Name'] == name, lat, data['Latitude'])
    data['Longitude'] = np.where(data['DBA Name'] == name, long, data['Longitude'])

In [56]:
print("Number of entries without coordinate data: " + str(len(data[data["Latitude"].apply(pd.isnull)])))
data[data["Latitude"].apply(pd.isnull)]

Number of entries without coordinate data: 0


,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude


### Zip
We supplement the zip codes by finding where the coordinates belong in a GeoJSON boundary map for zip codes.

In [131]:
data[data['Zip'].apply(pd.isnull)].sample(10)

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
194571,67807,SUN FOOD MART,2009479.0,Grocery Store,Risk 3 (Low),3601 W CERMAK RD,NaN,2010-01-11T00:00:00.000,License Re-Inspection,Pass,NaN,41.851553,-87.715059
174707,525228,THE LUNCH MACHINE INC,2079077.0,Mobile Food Dispenser,Risk 2 (Medium),1319 W WILSON AVE,NaN,2011-02-16T00:00:00.000,License,Fail,NaN,41.965212,-87.663260
123931,1106878,FILIPINO AMERICAN COUNCIL,2225427.0,Golden Diner,Risk 1 (High),1332 W DRIVING PARK ROAD BSMT,NaN,2013-09-26T00:00:00.000,Canvass,Out of Business,NaN,NaN,NaN
20750,2200428,LEO'S FOOD AND LIQUOR,2535068.0,Liquor,Risk 3 (Low),4471 W LAWRENCE AVE,NaN,2018-08-02T00:00:00.000,License,Not Ready,NaN,41.967993,-87.740726
120745,1228307,MCCORNICK PLACE,1932251.0,Restaurant,Risk 1 (High),2301 S INDIANA,NaN,2013-11-14T00:00:00.000,Canvass,Out of Business,NaN,41.851047,-87.622093
190589,60405,CASA CENTRAL LA POSADA,0.0,Restaurant,Risk 1 (High),,NaN,2010-04-05T00:00:00.000,Canvass,Pass,37. TOILET ROOM DOORS SELF CLOSING: DRESSING R...,NaN,NaN
1061,2312774,CHICAGO COLLEGIATE CHARTER,3846104.0,School,Risk 1 (High),10909 S COTTAGE GROVE AVE,NaN,2019-09-24T00:00:00.000,Canvass Re-Inspection,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.696087,-87.608945
135206,1114725,BURGER KING #1310,2232344.0,Restaurant,Risk 2 (Medium),2320-2322 S MICHIGAN AVE,NaN,2013-03-18T00:00:00.000,License,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.850454,-87.623891
84499,1566296,PARKWEST LIQUORS AND SMOKE SHOP,2354157.0,Liquor,Risk 3 (Low),2570 N LINCOLN AVE,NaN,2015-08-17T00:00:00.000,License Re-Inspection,Pass,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.928597,-87.653388
174129,525238,THE LUNCH MACHINE INC,2079077.0,Mobile Food Dispenser,Risk 2 (Medium),1319 W WILSON AVE,NaN,2011-03-01T00:00:00.000,License Re-Inspection,Fail,NaN,41.965212,-87.663260


In [133]:
map_zip = gpd.read_file('data/chicago_zip.geojson')
missing_zip = data[data['Zip'].apply(pd.isnull)].copy()
for index, lat, long in zip(missing_zip.index, missing_zip['Latitude'], missing_zip['Longitude']):
    for zip_code, mpoly in zip(map_zip.zip, map_zip.geometry): 
        for poly in mpoly:
            if poly.contains(Point(long,lat)): # Points in polygons have inverse lat, long
                missing_zip.at[index,'Zip'] = zip_code
data.update(missing_zip)

print("Number of entries without zip data: " + str(len(data[data['Zip'].apply(pd.isnull)]))
data[data['Zip'].apply(pd.isnull)]

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
69601,1763245.0,EAT N RUN CHICKEN AND FISH,2442868.0,Restaurant,Risk 1 (High),,NaN,2016-04-19T00:00:00.000,License,Not Ready,NaN,NaN,NaN
123931,1106878.0,FILIPINO AMERICAN COUNCIL,2225427.0,Golden Diner,Risk 1 (High),1332 W DRIVING PARK ROAD BSMT,NaN,2013-09-26T00:00:00.000,Canvass,Out of Business,NaN,NaN,NaN
190589,60405.0,CASA CENTRAL LA POSADA,0.0,Restaurant,Risk 1 (High),,NaN,2010-04-05T00:00:00.000,Canvass,Pass,37. TOILET ROOM DOORS SELF CLOSING: DRESSING R...,NaN,NaN


### License number
We decide that license numbers are not important for our research. (???)

In [141]:
data[data['License #'].apply(pd.isnull)].sample(10)

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
154371,1214242.0,GOD'S BATTLE AXE PRAYER ACADEMY,NaN,CHURCH/DAY CARE,Risk 1 (High),6969 N WESTERN AVE,60645.0,2012-03-19T00:00:00.000,Canvass Re-Inspection,Pass,NaN,42.008303,-87.690005
65373,1933084.0,ST. DEMETRIOS GREEK ORTHODOX CHURCH,NaN,Special Event,Risk 2 (Medium),2727 W WINONA ST,60625.0,2016-06-20T00:00:00.000,Canvass,Pass,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.974653,-87.697529
147647,1188285.0,ST. DEMETRIOS GREEK ORTHODOX CHURCH,NaN,Special Event,Risk 2 (Medium),2727 W WINONA ST,60625.0,2012-07-25T00:00:00.000,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.974653,-87.697529
22844,2181316.0,ST. DEMETRIOS GREEK ORTHODOX CHURCH,NaN,Special Event,Risk 2 (Medium),2727 W WINONA ST,60625.0,2018-06-13T00:00:00.000,Canvass,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.974653,-87.697529
110647,1459918.0,ST. DEMETRIOS GREEK ORTHODOX CHURCH,NaN,Special Event,Risk 2 (Medium),2727 W WINONA ST,60625.0,2014-05-20T00:00:00.000,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.974653,-87.697529
85255,1561809.0,ST. DEMETRIOS GREEK ORTHODOX CHURCH,NaN,Special Event,Risk 2 (Medium),2727 W WINONA ST,60625.0,2015-08-04T00:00:00.000,Canvass,Pass,NaN,41.974653,-87.697529
154583,679787.0,GOD'S BATTLE AXE PRAYER ACADEMY,NaN,CHURCH/DAY CARE,Risk 1 (High),6969 N WESTERN AVE,60645.0,2012-03-14T00:00:00.000,Canvass Re-Inspection,Fail,"9. WATER SOURCE: SAFE, HOT & COLD UNDER CITY P...",42.008303,-87.690005
156097,670328.0,GOD'S BATTLE AXE PRAYER ACADEMY,NaN,CHURCH/DAY CARE,Risk 1 (High),6969 N WESTERN AVE,60645.0,2012-02-09T00:00:00.000,Canvass,Fail,"11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D...",42.008303,-87.690005
113188,1152076.0,ARGENTINA FOODS,NaN,Grocery Store,Risk 2 (Medium),4500 S WOOD ST,60609.0,2014-04-10T00:00:00.000,Canvass,Out of Business,NaN,41.812074,-87.670247
176442,417318.0,AVALON COMMUNITY CHURCH/FREEDOM HOME ACADEMY,NaN,CHURCH/AFTER SCHOOL PROGRAM,Risk 1 (High),8100 S DANTE AVE,60619.0,2011-01-10T00:00:00.000,Short Form Complaint,Fail,NaN,41.747711,-87.589539


### Facility Type

In [146]:
data[data['Facility Type'].apply(pd.isnull)].sample(10)

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
185595,277319.0,LA VIDA,30802.0,NaN,Risk 1 (High),1248 W GEORGE ST,60657.0,2010-07-12T00:00:00.000,Canvass,Out of Business,NaN,41.934455,-87.660704
143433,1285763.0,LOS PINOS SUPERMERCADO,1595102.0,NaN,Risk 1 (High),3001 W DIVERSEY AVE,60647.0,2012-10-09T00:00:00.000,Canvass,Out of Business,NaN,41.931949,-87.702580
140064,1198610.0,LA TAVERNETTA CUCINA ITALIANA,1248448.0,NaN,Risk 1 (High),3023 N BROADWAY,60657.0,2012-12-11T00:00:00.000,Canvass,Out of Business,NaN,41.937201,-87.644158
128920,1106655.0,ZAND CAFE,1958736.0,NaN,Risk 1 (High),6764 N SHERIDAN RD,60626.0,2013-07-03T00:00:00.000,Canvass,Out of Business,NaN,42.005424,-87.661046
99992,1385760.0,"JIMMY'S MINI MART,INC.",2363323.0,NaN,Risk 2 (Medium),4805 W MADISON ST,60644.0,2014-11-06T00:00:00.000,License,No Entry,NaN,41.880385,-87.745544
129755,1155099.0,SUPERMERCADO JALPITA INC.,1965145.0,NaN,Risk 3 (Low),9800 S ESCANABA AVE,60617.0,2013-06-18T00:00:00.000,Canvass,Out of Business,NaN,41.717202,-87.553906
159286,539547.0,GOLD COAST DOGS,33986.0,NaN,Risk 3 (Low),11601 W TOUHY AVE,60666.0,2011-12-02T00:00:00.000,Canvass,Out of Business,NaN,42.008536,-87.914428
140209,1300838.0,THAI BINH CORP,25129.0,NaN,Risk 3 (Low),1113 -01115 W ARGYLE ST,60640.0,2012-12-07T00:00:00.000,Canvass,Out of Business,NaN,41.973156,-87.658315
184168,284216.0,KOSTNER FOOD MART INC,78349.0,NaN,Risk 3 (Low),24 S KOSTNER AVE,60624.0,2010-08-11T00:00:00.000,Canvass,Out of Business,NaN,41.879763,-87.735594
97991,1512796.0,ANTONIO'S SHELL SERVICE,1243338.0,NaN,Risk 3 (Low),2801 W FULLERTON AVE,60647.0,2014-12-09T00:00:00.000,Canvass,Out of Business,NaN,41.924697,-87.697472


### Risk

In [149]:
data[data['Risk'].apply(pd.isnull)].sample(10)

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
128543,1296466.0,QUICK STOP FOOD MART,2271408.0,NaN,NaN,2224 E 79TH ST,60617.0,2013-07-12T00:00:00.000,License,No Entry,NaN,41.751792,-87.570034
102311,1497320.0,WHAT'S IN THE BAG?,2359248.0,NaN,NaN,1767 W 95TH ST,60643.0,2014-09-30T00:00:00.000,License,Fail,NaN,41.721038,-87.667398
182244,413465.0,JW MARRIOT CHICAGO,0.0,Restaurant,NaN,151 W Adams ST,60603.0,2010-09-15T00:00:00.000,Pre-License Consultation,Fail,NaN,41.879290,-87.632363
171,2316086.0,UNIVERSITY CENTER PROPERTY LLC,2694616.0,NaN,NaN,525 S STATE ST,60605.0,2019-10-18T00:00:00.000,License,Not Ready,NaN,41.875179,-87.627474
177865,470238.0,CHICAGO O'HARE GARDEN HOTEL,1380014.0,unlicensed facility,NaN,8201 W HIGGINS RD,60631.0,2010-12-03T00:00:00.000,Consultation,Pass,NaN,41.985652,-87.832152
192096,58348.0,UNIVERSITY OF AESTHETICS/TOUCH CLINIC,0.0,NaN,NaN,1357 N MILWAUKEE AVE,60622.0,2010-03-04T00:00:00.000,Short Form Complaint,Pass,NaN,41.906596,-87.671323
135810,1322185.0,CONGRESS THEATRE,2196827.0,NaN,NaN,2135 N MILWAUKEE AVE,60647.0,2013-03-07T00:00:00.000,License,Fail,NaN,41.920055,-87.692462
158082,659843.0,LAS BRISAS DEL SUR INC,2134071.0,Mobile Food Dispenser,NaN,2637 S THROOP ST FL,60608.0,2012-01-03T00:00:00.000,License,Fail,NaN,41.844671,-87.654962
176992,487855.0,DUNKIN DONUTS,0.0,NaN,NaN,2640 W DIVERSEY AVE,60647.0,2010-12-23T00:00:00.000,Complaint,Business Not Located,NaN,41.780221,-87.644765
188703,233830.0,PONCE PASEO BROROQUA GROCERY STORE,0.0,NaN,NaN,2706 W DIVERSEY AVE,60647.0,2010-05-12T00:00:00.000,Complaint,Fail,NaN,41.932217,-87.695489


### Inspection Type

In [9]:
data[data['Inspection Type'].apply(pd.isnull)]

,Inspection ID,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
63843,1946612,ARAMARL-FRESHII,2470318.0,Restaurant,Risk 1 (High),2650 N CLARK ST,60614.0,2016-07-14T00:00:00.000,NaN,Pass,NaN,41.930833,-87.644135


### Violations
Since there is no way we can know which violations have been done, we fill null values with "No data".

In [9]:
data[data['Violations'].apply(pd.isnull)].sample(10)
data['Violations'] = data['Violations'].fillna('No data')

# Information about columns

## Facility Type
From https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF:

Type of facility: Each establishment is described by one of the following: bakery, banquet hall, candy store, caterer, coffee shop, day care center (for ages less than 2), day care center (for ages 2 – 6), day care center (combo, for ages less than 2 and 2 – 6 combined), gas station, Golden Diner, grocery store, hospital, long term care center(nursing home), liquor store, mobile food dispenser, restaurant, paleteria, school, shelter, tavern, social club, wholesaler, or Wrigley Field Rooftop. 

In [244]:
facility_types = data['Facility Type'].drop_duplicates().reset_index(drop=True).apply(str)
valid_facility_types = ['bakery', 'banquet hall', 'candy', 'caterer', 'coffee shop', 'daycare', 'gas station', 'golden diner', 'grocery store', 'hospital', 'long term', 'liquor store', 'mobile food', 'restaurant', 'paleteria', 'school', 'shelter', 'tavern', 'social club', 'wholesaler', 'wrigley field rooftop']
print(facility_types[facility_types.apply(str.lower).map(lambda x: any([y in x for y in valid_facility_types]))])
print(facility_types[facility_types.apply(str.lower).map(lambda x: not any([y in x for y in valid_facility_types]))])
# What to do? There are sooo many different facilities.

0                           Restaurant
2                        Grocery Store
3      Daycare Above and Under 2 Years
4                Daycare (2 - 6 Years)
6                 Mobile Food Preparer
                    ...               
481                       candy/gelato
482                        CANDY MAKER
484                      TAVERN/LIQUOR
485                              Candy
487                   WHOLESALE BAKERY
Name: Facility Type, Length: 145, dtype: object
1      Children's Services Facility
5                          Catering
7                            Liquor
10      CHILDRENS SERVICES FACILITY
11                              nan
                   ...             
479                    SOUP KITCHEN
480                    hooka lounge
483                       religious
486                       incubator
488                      Kids Cafe'
Name: Facility Type, Length: 344, dtype: object


## Inspection types
From https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF:

Inspection type: An inspection can be one of the following types: 
* canvass, the most common type of inspection performed at a frequency relative to the risk of the establishment; 
* consultation, when the inspection is done at the request of the owner prior to the opening of the establishment; complaint, when  the inspection is done in response to a complaint against the establishment; 
* license, when the inspection is done as a requirement for the establishment to receive its license to operate; 
* suspect food poisoning, when the inspection is done in response to one or more persons claiming to have gotten ill as a result of eating at the establishment (a specific type of complaint-based inspection); 
* task-force inspection, when an inspection of a bar or tavern is done. 
Re-inspections can occur for most types of these inspections and are indicated as such.

In [226]:
inspection_types = data['Inspection Type'].drop_duplicates().reset_index(drop=True).apply(str)
valid_inspection_types = ['canvass', 'consultation', 'license', 'complaint', 'food poisoning', 'task']
pd.set_option('display.max_rows', 108)
print(inspection_types[inspection_types.apply(str.lower).map(lambda x: any([y in x for y in valid_inspection_types]))])
print(inspection_types[inspection_types.apply(str.lower).map(lambda x: not any([y in x for y in valid_inspection_types]))])
pd.reset_option('display.max_rows')

# What to do? We could manually annotate them since there are not too many. 
# We also need to figure out what a combination of types mean, for example "License-Task Force"
# Also, there are a few things that are not correctly classified using only keywords, for example "OWNER SUSPENDED OPERATION/LICENSE" will be classified as license

0                          License Re-Inspection
1                        Complaint Re-Inspection
2                                        License
3                                        Canvass
4                          Canvass Re-Inspection
6                                      Complaint
7                           Short Form Complaint
8                       Suspected Food Poisoning
9         Suspected Food Poisoning Re-inspection
11                                  Consultation
15                                fire complaint
19                            License-Task Force
20                  Complaint-Fire Re-inspection
21                                Complaint-Fire
22                     Short Form Fire-Complaint
26                     LICENSE CANCELED BY OWNER
27             OWNER SUSPENDED OPERATION/LICENSE
28                          LICENSE CONSULTATION
29                          License consultation
30                        Task Force Liquor 1475
34                  

## Results
From https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF:

An inspection can pass, pass with conditions or fail. Establishments receiving a ‘pass’ were found to have no critical or serious violations (violation number 1-14 and 15-29, respectively). Establishments receiving a ‘pass with conditions’ were found to have critical or serious violations, but these were corrected during the inspection. Establishments receiving a ‘fail’ were found to have critical or serious violations that were not correctable during the inspection. An establishment receiving a ‘fail’ does not necessarily mean the establishment’s licensed is suspended. Establishments found to be out of business or not located are indicated as such. 

In [49]:
results = data['Results'].drop_duplicates().reset_index(drop=True).apply(str)
valid_results = ['pass','pass w/ conditions','fail','out of business', 'business not located']
print(results[results.apply(str.lower).map(lambda x: any([y == x for y in valid_results]))])
print(results[results.apply(str.lower).map(lambda x: not any([y == x for y in valid_results]))])
# Easy to add some more categories

0                    Fail
1                    Pass
3      Pass w/ Conditions
5         Out of Business
6    Business Not Located
Name: Results, dtype: object
2    Not Ready
4     No Entry
Name: Results, dtype: object


# Finding useful data in the Violations column
Violations seem to have the structure:
(&lt;violation number>. &lt;violation title> - &lt;violation comments> - | )+

According to https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF, critical violations are numbered 1-14, and serious violations are numbered 15-29. If these are found and fixed during the inspection, the inspection will pass with conditions. Otherwise it will fail.

In [41]:
data['Violations'][4]

'54. GARBAGE & REFUSE PROPERLY DISPOSED; FACILITIES MAINTAINED - Comments: OBSERVED THE OUTSIDE ENCLOSED REFUSE AREA NOT MAINTAIN. INSTRUCTED MANAGER TO REMOVE AND CLEAN AREA TO PREVENT HARBORAGE. | 55. PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN - Comments: NOTED WATER STAINED CEILING TILES IN THE KIDS WASHROOM. MUST REPLACE AND MAINTAIN. | 56. ADEQUATE VENTILATION & LIGHTING; DESIGNATED AREAS USED - Comments: MUST CLEAN THE HOOD ABOVE THE COOKING EQUIPMENT TO REMOVE DUST.'

In [28]:
re_num = re.compile(r'([0-9]+)\.')
violation_numbers = data['Violations'].apply(re_num.findall).apply(lambda xs: list(map(lambda x: int(x), xs)))
critical_violations = range(1,14+1)
serious_violations = range(15, 29+1)
violation_numbers
# Maybe we can actually connect violation numbers to result "Out of business"
# See under "Under what conditions will a food establishment get shut down?" https://dchealth.dc.gov/service/understanding-food-establishment-inspections
# While this is for Washington DC, I think it could apply to us too
# Also see "Results" in https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF

0                   [3, 5, 25, 50, 74, 53, 53, 56, 58]
1                                                   []
2                                                   []
3                                         [10, 55, 56]
4                                         [54, 55, 56]
                              ...                     
194899                                              []
194900                            [18, 32, 34, 35, 36]
194901                                              []
194902    [18, 19, 32, 33, 34, 35, 36, 37, 38, 39, 41]
194903                                              []
Name: Violations, Length: 194708, dtype: object

In [29]:
violations_severity = violation_numbers.apply(lambda xs: list(map(lambda x: 'critical' if x in critical_violations else 'serious' if x in serious_violations else 'non-serious', xs)))
violations_severity

0         [critical, critical, serious, non-serious, non...
1                                                        []
2                                                        []
3                      [critical, non-serious, non-serious]
4                   [non-serious, non-serious, non-serious]
                                ...                        
194899                                                   []
194900    [serious, non-serious, non-serious, non-seriou...
194901                                                   []
194902    [serious, serious, non-serious, non-serious, n...
194903                                                   []
Name: Violations, Length: 194708, dtype: object

# Supplementing the data

## Income by zip code (actually, anything by zip code if we want)
https://towardsdatascience.com/getting-census-data-in-5-easy-steps-a08eeb63995d

Attribution statement: This product uses the Census Bureau Data API but is not endorsed or certified by the Census Bureau.

Zip code 60666 is the zip code for O'Hare International Airport.
Zip code 60627 is the zip code for Golden Gate Park.
These do not have a household median income since nobody lives there.

In [ ]:
income_by_zip_address = "https://api.census.gov/data/%s/acs/acs5?key=%s&get=%s&for=zip%%20code%%20tabulation%%20area:%s" 
year = "2017" # 2009-2017
api_key = "" # Request here: https://www.census.gov/developers/
income_variable = "B19013_001E"
zip_codes = data['Zip'].drop_duplicates()

for zip_code in zip_codes:
    if pd.isnull(zip_code):
        continue
        
    called_api = income_by_zip_address % (year, api_key, income_variable, int(zip_code))
    response = requests.get(called_api)
    if response.status_code == 200:
        formatted_response = json.loads(response.text)
        print(formatted_response)
    else:
        print('Could not find income for zip code: ' + str(int(zip_code)))
    # add to some list or something, also exception handling
    

In [169]:
# chicago_map = gpd.read_file('data/chicago_zip.shp') # https://www.kaggle.com/threadid/chicago-shape-files

In [170]:
# ax = chicago_map.plot()
# g_data.plot(ax=ax, color='red')

# Projecting information on a map

## Coordinates
We want to be able to visualize our dataset on a map based on coordinates. This is done by using Folium.

In [65]:
data_names = data.groupby('DBA Name').first()
data_names = data_names[data_names['Zip'].apply(pd.isnull)]
print(len(data_names))

19


In [72]:
middle_lat = data_names.Latitude.median()
middle_lon = data_names.Longitude.median()
ch_map = folium.Map(location = [middle_lat, middle_lon], zoom_start=10)
locations = folium.map.FeatureGroup()
lats = data_names.Latitude
lons = data_names.Longitude
names = data_names.index


for lat, lon, name in list(zip(lats, lons, names))[0:10]:
    if np.isnan(lat) or np.isnan(lon):
        continue
    folium.Marker(
        location =  [lat, lon],
        tooltip = name,
        icon = folium.Icon(color='green', icon='info-sign')
    ).add_to(ch_map)
    
ch_map.add_child(locations)
print("Done")

Done


## Zip code areas
We use the zip code boundaries in a GeoJSON format from https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-ZIP-Codes/gdcf-axmw. We then simply overlay them on the map.

In [73]:
ch_map = folium.Map(location = [middle_lat, middle_lon], zoom_start=10)

data_zip = data.groupby('Zip').count().reset_index()[['Zip', 'Risk']].rename(columns={'Risk':'Count'})
data_zip['Zip'] = data_zip['Zip'].apply(int).apply(str)
map_zip = gpd.read_file('data/chicago_zip.geojson')
map_zip = map_zip.merge(data_zip, left_on='zip', right_on='Zip')
zip_geojson = folium.GeoJson(map_zip)

ch_choropleth = folium.Choropleth(
    geo_data=map_zip,
    name='choropleth',
    data=data_zip,
    columns=['Zip', 'Count'],
    key_on='feature.properties.zip',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Number of inspections per Zip',
)

folium.GeoJsonTooltip(["Count"]).add_to(ch_choropleth.geojson)

ch_choropleth.add_to(ch_map)

#folium.LayerControl().add_to(ch_map)
ch_map